In [1]:
from pyomo.environ import (
    ConcreteModel,
    Objective,
    Expression,
    value,
    Var,
    Param,
    Constraint,
    Set,
    Var,
    Block,
    SolverFactory,
    TransformationFactory,
    assert_optimal_termination,
    check_optimal_termination,
    log,
    log10,
    units as pyunits,
)

from pyomo.network import Port
from idaes.core import FlowsheetBlock
from idaes.core.solvers.get_solver import get_solver

from idaes.core.util.model_statistics import *
from idaes.core.util.scaling import *

from idaes.core import UnitModelCostingBlock

from watertap.core.zero_order_costing import ZeroOrderCosting, _get_tech_parameters
from watertap.core.util.infeasible import *
from watertap.costing import WaterTAPCosting

import json
from os.path import join, dirname
from math import floor, ceil

import pytest

from io import StringIO
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from IPython.display import clear_output
from copy import deepcopy
from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock
from watertap.property_models.water_prop_pack import WaterParameterBlock
# from watertap_contrib.seto.costing import SETOZeroOrderCosting, SETOWaterTAPCosting

# from watertap_contrib.seto.solar_models.zero_order import PhotovoltaicZO
# from watertap_contrib.seto.energy import solar_energy
# from watertap_contrib.seto.core import SETODatabase, PySAMWaterTAP
from watertap_contrib.seto.unit_models.chemical_softening_0D_MH import ChemicalSoftening0D
# from watertap_contrib.seto.property_models.chem_softening_prop_pack_MH import (
#     ChemSofteningParameterBlock,
#     ChemSofteningStateBlock
# )

from watertap_contrib.seto.property_models.basic_water_properties import (
    BasicWaterParameterBlock,
)
# from watertap_contrib.seto.costing.solar import photovoltaic

solver = get_solver()

def get_ion_config(ions):
    #neutral_solutes = ["TSS", "TDS", "TOC", "NH3", "SiO2"]
    if not isinstance(ions, list):
        ions = [ions]
    #ions = ions + neutral_solutes

    mw_data = {
        "Na_+": 23e-3,
        "Ca_2+": 40e-3,
        "Cl_-": 35e-3,
        "Mg_2+": 24e-3,
        "SO4_2-": 96e-3,
        "NH3": 17.03e-3,
        "SiO2": 60.08e-3,
        "HCO3_-": 61.02e-3,
        "Alkalinity_2-": 31.01736e-3,
        "CO3_2-": 60.01e-3,
        "CO2": 44e-3,
    }
    charge_data = {
        "Na_+": 1,
        "Ca_2+": 2,
        "Cl_-": -1,
        "Mg_2+": 2,
        "SO4_2-": -2,
        "HCO3_-": -1,
        "CO3_2-": -2,
        "Alkalinity_2-": -2,
    }
    ion_config = {
        "solute_list": [],
        "mw_data": {"H2O": 18e-3},
        "mw_data": {},
        "charge": {},
    }

    for ion in ions:
        ion_config["solute_list"].append(ion)

        if ion in charge_data.keys():
            ion_config["charge"][ion] = charge_data[ion]
        if ion in mw_data.keys():
            ion_config["mw_data"][ion] = mw_data[ion]
    return ion_config


In [2]:
component_list = ["Ca_2+","Mg_2+","SiO2","Alkalinity_2-","TSS"]
input_config_dict = get_ion_config(component_list)
input_config_dict

{'solute_list': ['Ca_2+', 'Mg_2+', 'SiO2', 'Alkalinity_2-', 'TSS'],
 'mw_data': {'Ca_2+': 0.04,
  'Mg_2+': 0.024,
  'SiO2': 0.06008,
  'Alkalinity_2-': 0.03101736},
 'charge': {'Ca_2+': 2, 'Mg_2+': 2, 'Alkalinity_2-': -2}}

In [3]:
ca_in = 0.075 * pyunits.kg / pyunits.m**3  # g/L = kg/m3
mg_in = 0.0061 * pyunits.kg / pyunits.m**3  # g/L = kg/m3
sio2_in = 0.054 * pyunits.kg / pyunits.m**3  # g/L = kg/m3
alk_in = 0.196 * pyunits.kg / pyunits.m**3  # g/L = kg/m3

q_in = 50000 * pyunits.m**3 / pyunits.day  # m3/d
rho = 1000 * pyunits.kg / pyunits.m**3

pH_in = 12
temp_in = 293.15  # K

ca_mass_flow = pyunits.convert((ca_in * q_in), to_units=pyunits.kg / pyunits.s)()
mg_mass_flow = pyunits.convert((mg_in * q_in), to_units=pyunits.kg / pyunits.s)()
sio2_mass_flow = pyunits.convert((sio2_in * q_in), to_units=pyunits.kg / pyunits.s)()
h2o_mass_flow = pyunits.convert((rho * q_in), to_units=pyunits.kg / pyunits.s)()



In [7]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
# m.fs.properties = ChemSofteningParameterBlock(**input_config_dict)
m.fs.properties =  BasicWaterParameterBlock(solute_list=component_list)
m.fs.soft = soft = ChemicalSoftening0D(
    property_package=m.fs.properties, silica_removal= True ,softening_procedure_type= 'single_stage_lime'
)

prop_in = m.fs.soft.properties_in[0]
prop_out = m.fs.soft.properties_out[0]
prop_waste = m.fs.soft.properties_waste[0]

print(f"DOF = {degrees_of_freedom(m)}")


case 2
Mg calculated
DOF = 12


In [5]:
# prop_in.flow_mass_comp["Ca_2+"].fix(ca_mass_flow)
# set_scaling_factor(prop_in.flow_mass_comp["Ca_2+"], 100)
# prop_in.flow_mass_comp[ "Mg_2+"].fix(mg_mass_flow)
# set_scaling_factor(prop_in.flow_mass_comp[ "Mg_2+"], 100)
# prop_in.flow_mass_comp[ "SiO2"].fix(sio2_mass_flow)
# set_scaling_factor(prop_in.flow_mass_comp[ "SiO2"], 100)
# prop_in.flow_mass_comp["H2O"].fix(h2o_mass_flow)
# set_scaling_factor(prop_in.flow_mass_comp[ "H2O"], 1e-2)

# prop_out.flow_vol.fix(0.1*q_in)
# prop_waste.flow_vol.fix(0.9*q_in)

In [5]:
prop_in.params.component_set.data()

('Ca_2+', 'Mg_2+', 'SiO2', 'Alkalinity_2-', 'TSS')

In [8]:
prop_in.flow_vol.fix(q_in)

# prop_in.alkalinity.fix(0.195)
# prop_in.pH.fix(pH_in)

prop_in.conc_mass_comp["Ca_2+"].fix(ca_in)
prop_in.conc_mass_comp["Mg_2+"].fix(mg_in)
prop_in.conc_mass_comp["SiO2"].fix(sio2_in)
prop_in.conc_mass_comp["Alkalinity_2-"].fix(alk_in)
prop_in.conc_mass_comp["TSS"].fix(alk_in)

#prop_in.flow_mass_comp["H2O"].fix(h2o_mass_flow)

set_scaling_factor(prop_in.conc_mass_comp["Ca_2+"],100)
set_scaling_factor(prop_in.conc_mass_comp["Mg_2+"],100)
set_scaling_factor(prop_in.conc_mass_comp["SiO2"],100)
set_scaling_factor(prop_in.conc_mass_comp["Alkalinity_2-"],100)
set_scaling_factor(prop_in.conc_mass_comp["TSS"],100)

soft.retention_time_mixer.fix()
soft.retention_time_floc.fix()
soft.retention_time_sed.fix()
soft.retention_time_recarb.fix()
soft.frac_vol_recovery.fix()
soft.CO2_CaCO3.fix()
m.fs.soft.initialize()

print(f"DOF = {degrees_of_freedom(m)}")


2023-03-03 12:42:51 [INFO] idaes.init.fs.soft: Initialization Step 1a Complete.
2023-03-03 12:42:51 [INFO] idaes.init.fs.soft.properties_out: State Released.
2023-03-03 12:42:51 [INFO] idaes.init.fs.soft: Initialization Step 1b Complete.
2023-03-03 12:42:52 [INFO] idaes.init.fs.soft.properties_waste: State Released.
2023-03-03 12:42:52 [INFO] idaes.init.fs.soft: Initialization Step 1c Complete.
2023-03-03 12:42:52 [INFO] idaes.init.fs.soft: Initialization Step 2 optimal - Optimal Solution Found.
2023-03-03 12:42:52 [INFO] idaes.init.fs.soft.properties_in: State Released.
2023-03-03 12:42:52 [INFO] idaes.init.fs.soft: Initialization Complete: optimal - Optimal Solution Found
DOF = 0


In [9]:
print(value(prop_in.flow_vol) *9.2*value(prop_in.conc_mass_comp["SiO2"]))
print(soft.MgCl2_dosing.value)


0.2874999999999999
0.2874999999999999


In [15]:
# print(value(prop_in.flow_mass_comp['Ca_2+']))
# print(value(prop_out.flow_mass_comp['Ca_2+']))
# print(value(prop_out.flow_vol))
# print(value(prop_in.conc_mass_comp['Ca_2+']))
# print(value(prop_out.conc_mass_comp['Ca_2+']))

# prop_out.conc_mass_comp.display()
# prop_waste.conc_mass_comp.display()

prop_waste.flow_mass_comp['H2O'].value

5.7870370370370905

In [7]:

print(f"DOF = {degrees_of_freedom(m)}")

DOF = 0


In [8]:
results = solver.solve(m)

print(f"DOF = {degrees_of_freedom(m)}")
print(results.solver.termination_condition.swapcase())
print_infeasible_constraints(m)


DOF = 0
OPTIMAL
